<a href="https://colab.research.google.com/github/nirajlondhe8/aiml/blob/main/Algo_eval_suprise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import files
import zipfile
import os

# Upload the ml-latest-small.zip file
uploaded = files.upload()

# Unzip the uploaded file
zip_filename = 'ml-latest-small.zip'
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall('ml-latest-small')

# List all files in the extracted directory
extracted_files = []
for root, dirs, files in os.walk('ml-latest-small'):
    for file in files:
        extracted_files.append(os.path.join(root, file))

# Display the extracted files
for file in extracted_files:
    print(file)


Saving ml-latest-small.zip to ml-latest-small (1).zip
ml-latest-small/ml-latest-small/movies.csv
ml-latest-small/ml-latest-small/tags.csv
ml-latest-small/ml-latest-small/ratings.csv
ml-latest-small/ml-latest-small/README.txt
ml-latest-small/ml-latest-small/links.csv
ml-latest-small/__MACOSX/._ml-latest-small
ml-latest-small/__MACOSX/ml-latest-small/._movies.csv
ml-latest-small/__MACOSX/ml-latest-small/._ratings.csv
ml-latest-small/__MACOSX/ml-latest-small/._README.txt
ml-latest-small/__MACOSX/ml-latest-small/._links.csv
ml-latest-small/__MACOSX/ml-latest-small/._tags.csv


In [2]:
import os

# List the contents of the extracted folder
os.listdir('ml-latest-small')


['ml-latest-small', '__MACOSX']

In [4]:
!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357285 sha256=7bfe85f5103d1cfb778120db5e15973d8290f3b6fc1d697390a3772308013d47
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [8]:

import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate
from surprise import KNNBasic, SVD, NMF, SlopeOne, CoClustering

# Load the dataset
file_path = 'ml-latest-small/ml-latest-small/ratings.csv'
data = pd.read_csv(file_path)

# Inspect the first few rows
print(data.head())

# Use the Surprise Reader to define the rating scale
reader = Reader(rating_scale=(0.5, 5))

# Load the data into Surprise's format
dataset = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader)


   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [9]:
# Define the algorithms to compare
algorithms = {
    'KNNBasic': KNNBasic(),
    'SVD': SVD(),
    'NMF': NMF(),
    'SlopeOne': SlopeOne(),
    'CoClustering': CoClustering()
}

# Dictionary to store results
results = {}

# Evaluate each algorithm using cross-validation
for algo_name, algo in algorithms.items():
    print(f"Evaluating {algo_name}...")
    result = cross_validate(algo, dataset, measures=['RMSE'], cv=5, verbose=True)
    results[algo_name] = result

# Display RMSE results for each algorithm
for algo_name, result in results.items():
    print(f"\nAlgorithm: {algo_name}")
    print(f"Mean RMSE: {result['test_rmse'].mean():.4f}")
    print(f"Standard Deviation: {result['test_rmse'].std():.4f}")


Evaluating KNNBasic...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9399  0.9443  0.9585  0.9503  0.9495  0.9485  0.0063  
Fit time          0.24    0.14    0.13    0.13    0.13    0.15    0.04    
Test time         2.23    1.29    1.38    1.28    1.29    1.50    0.37    
Evaluating SVD...
Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8745  0.8721  0.8750  0.8699  0.8770  0.8737  0.0025  
Fit time          2.20    1.45    1.44   